In [3]:
import os

# Set environment variables
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"] = "Parent_Child"

# Verify that the environment variables are set
print("Environment Variables Set:")
print(f'LANGSMITH_TRACING: {os.getenv("LANGSMITH_TRACING")}')
print(f'LANGSMITH_ENDPOINT: {os.getenv("LANGSMITH_ENDPOINT")}')
print(f'LANGSMITH_PROJECT: {os.getenv("LANGSMITH_PROJECT")}')

Environment Variables Set:
LANGSMITH_TRACING: true
LANGSMITH_ENDPOINT: https://api.smith.langchain.com
LANGSMITH_PROJECT: Parent_Child


In [4]:
import langchain
import pandas as pd
import asyncio
import tqdm
from langchain_community.document_loaders import PyPDFLoader
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain
from langsmith import traceable
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.vectorstores import Chroma
from langsmith import traceable
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

In [5]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ········


## LLM and Embedding Model

In [6]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens = 4000
)

In [7]:
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large"
)

In [8]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

# Load the FAISS vector store for summaries
summary_vectorstore = FAISS.load_local("summary_vectorstore_new", OpenAIEmbeddings(model="text-embedding-3-large"), allow_dangerous_deserialization=True)
detailed_vectorstore = FAISS.load_local("detailed_vectorstore_new",  OpenAIEmbeddings(model="text-embedding-3-large"), allow_dangerous_deserialization=True)

/var/folders/8_/66zkglc51d973l0mk64lcrkc0000gn/T/ipykernel_4588/1972013706.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  summary_vectorstore = FAISS.load_local("summary_vectorstore_new", OpenAIEmbeddings(model="text-embedding-3-large"), allow_dangerous_deserialization=True)


In [9]:
@traceable
def retrieve_hierarchical(query, summary_vectorstore, detailed_vectorstore, k_summaries=3, k_chunks=5):
    """
    Performs a hierarchical retrieval using the query.

    Args:
        query: The search query.
        summary_vectorstore: The vector store containing document summaries.
        detailed_vectorstore: The vector store containing detailed chunks.
        k_summaries: The number of top summaries to retrieve.
        k_chunks: The number of detailed chunks to retrieve per summary.

    Returns:
        A list of dictionaries, each containing:
            {
                "summary": <Document>, 
                "relevant_chunks": [<Document>, ...]
            }
    """
    # Retrieve top summaries
    top_summaries = summary_vectorstore.similarity_search(query, k=k_summaries)

    # Prepare a list to hold the final results
    retrieved_results = []
    
    for summary in top_summaries:
        # For each summary, retrieve relevant detailed chunks from the same page
        page_number = summary.metadata["page"]  # now guaranteed to be int

        # Create a filter to only retrieve chunks from the same page
        def page_filter(metadata):
            # Also ensure we read the page as int
            return metadata.get("page", None) == page_number

        # Get the top k_chunks from the detailed vector store
        page_chunks = detailed_vectorstore.similarity_search(
            query,
            k=k_chunks,
            filter=page_filter
        )

        retrieved_results.append({
            "summary": summary,
            "relevant_chunks": page_chunks
        })
    
    return retrieved_results


## Setting up the RAG pipeline 

In [10]:
def extract_summary(result):
    """
    Extracts relevant chunks from the hierarchical retrieval result.
    """
    return [chunk for entry in result for chunk in entry["summary"]]

In [11]:
def extract_relevant_chunks(result):
    """
    Extracts relevant chunks from the hierarchical retrieval result.
    """
    return [chunk for entry in result for chunk in entry["relevant_chunks"]]

In [12]:
def create_summary_dataframe(doc_tuples):
    """
    Processes a list of document tuples and creates a structured DataFrame.
    
    Parameters:
        doc_tuples (list of tuples): List containing document attributes.
        
    Returns:
        pd.DataFrame: A DataFrame with document name, page number, and retrieved chunk.
    """
    docs = []
    current_doc = {}
    
    for key, value in doc_tuples:
        if key == 'id' and current_doc:
            docs.append(current_doc)  # Save the previous document
            current_doc = {}  # Reset for the next document
        current_doc[key] = value

    # Add the last document
    if current_doc:
        docs.append(current_doc)

    # Convert to DataFrame
    df = pd.DataFrame([
        {
            "Document Name": doc.get("metadata", {}).get("source", "Unknown Document"),
            "Page Number": doc.get("metadata", {}).get("page", "N/A"),
            "Retrieved Chunk": doc.get("page_content", "")[:1000] + "..."
        }
        for doc in docs
    ])

    return df

In [13]:
def create_chunk_dataframe(docs):
    """
    Creates a DataFrame from retrieved document details and scores.
    """
    return pd.DataFrame([
        {
            "Document Name": doc.metadata.get("source", "Unknown Document"),
            "Page Number": doc.metadata.get("page", "N/A"),
#             "Similarity Score": round(score, 4),  # Rounding for readability
            "Retrieved Chunk": doc.page_content[:1000] + "..."  # Show up to 1000 characters
        }
        for doc in docs
    ])

In [14]:
def display_document_dataframe(df, name="Document Details"):
    """
    Configures DataFrame display settings and displays it.
    """
    pd.set_option("display.max_colwidth", None)  # Do not truncate column content
    pd.set_option("display.width", 1000)  # Set a larger display width
    
    # Display the DataFrame
    display(df)

In [15]:
def setup_rag_pipeline():
    template = """"
    You are a helpful assistant that answers questions based on the provided context.
    Use the provided context to answer the question.
    Question: {input}
    Context: {context}
    Answer:
    """
    prompt = ChatPromptTemplate.from_template(template)
    
    # Setup RAG pipeline
    return (
        {"context": RunnablePassthrough(), "input": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

# Initialize the RAG chain with tracing
rag_chain = setup_rag_pipeline()

In [16]:
@traceable
def query_rag(question: str):
    """
    Query the RAG pipeline, retrieve relevant documents, and display their similarity scores in a table.
    """
    # Retrieve the top-k chunks with similarity scores
    retrieved_docs = retrieve_hierarchical(question, summary_vectorstore, detailed_vectorstore, k_summaries=3, k_chunks=5)
    
    extracted_summary = extract_summary(retrieved_docs)
    
    summary_df = create_summary_dataframe(extracted_summary)
    
    display_document_dataframe(summary_df)
    
    extracted_chunks = extract_relevant_chunks(retrieved_docs)
    
    chunk_df = create_chunk_dataframe(extracted_chunks)
    
    display_document_dataframe(chunk_df)
    
    # Extract contexts (raw document texts)
    retrieved_contexts = [doc.page_content for doc in extracted_chunks]
    
    
    # Run the RAG pipeline with the retrieved context
    answer = rag_chain.invoke({"input": question, "context": extracted_chunks})
    
    return {
        "answer": answer,
        "contexts": retrieved_contexts
    }


In [17]:
result = query_rag("Who is the CEO of the Company?")

,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,84,"The report, submitted under the Securities Exchange Act of 1934, features signatures from key executives and directors, including President and CEO Jen-Hsun Huang and CFO Colette M. Kress, affirming their roles as of February 21, 2024...."
1,NVIDIA_Report_2024.pdf,87,"Jen-Hsun Huang, CEO of NVIDIA, certifies the accuracy of the company's Annual Report on Form 10-K, affirming that it contains no misleading statements and fairly represents the financial condition. He outlines management's responsibilities for disclosure controls and internal financial reporting, and discloses any significant deficiencies or fraud. The certification is dated February 21, 2024...."
2,NVIDIA_Report_2024.pdf,83,"On February 21, 2024, NVIDIA Corporation submitted a report, signed by CEO Jen-Hsun Huang, in accordance with the Securities Exchange Act of 1934. The report includes a Power of Attorney designating Huang and Colette M.Kress as attorneys-in-fact to sign amendments and file related documents with the SEC...."


,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,84,"Pursuant to the requirements of the Securities Exchange Act of 1934, this report has been signed below by the following persons on behalf of the registrant andin the capacities and on the dates indicated.\nSignature Title Date\n/s/ JEN-HSUN HUANG President, Chief Executive Officer and Director(Principal Executive Officer) February 21, 2024\nJen-Hsun Huang \n/s/ COLETTE M. KRESS Executive Vice President and Chief Financial Officer(Principal Financial Officer) February 21, 2024\nColette M. Kress \n/s/ DONALD ROBERTSON Vice President and Chief Accounting Officer(Principal Accounting Officer) February 21, 2024\nDonald Robertson\n/s/ ROBERT BURGESS Director February 21, 2024\nRobert Burgess\n/s/ TENCH COXE Director February 21, 2024\nTench Coxe \n/s/ JOHN O. DABIRI Director February 21, 2024\nJohn O. Dabiri \n/s/ PERSIS DRELL Director February 21, 2024\nPersis Drell\n/s/ DAWN HUDSON Director February 21, 2024\nDawn Hudson\n/s/ HARVEY C. JONES Director February 21, 2024\nHarvey C. Jones..."
1,NVIDIA_Report_2024.pdf,84,"John O. Dabiri \n/s/ PERSIS DRELL Director February 21, 2024\nPersis Drell\n/s/ DAWN HUDSON Director February 21, 2024\nDawn Hudson\n/s/ HARVEY C. JONES Director February 21, 2024\nHarvey C. Jones \n/s/ MELISSA B. LORA Director February 21, 2024\nMelissa B. Lora\n/s/ MICHAEL MCCAFFERY Director February 21, 2024\nMichael McCaffery\n/s/ STEPHEN C. NEAL Director February 21, 2024\nStephen C. Neal\n/s/ MARK L. PERRY Director February 21, 2024\nMark L. Perry \n/s/ A. BROOKE SEAWELL Director February 21, 2024\nA. Brooke Seawell \n/s/ AARTI SHAH Director February 21, 2024\nAarti Shah\n/s/ MARK STEVENS Director February 21, 2024\nMark Stevens \n85..."
2,NVIDIA_Report_2024.pdf,87,"affect, the registrant's internal control over financial reporting; and\n5. The registrant's other certifying officer(s) and I have disclosed, based on our most recent evaluation of internal control over financial reporting,\nto the registrant's auditors and the audit committee of the registrant's board of directors (or persons performing the equivalent functions): \n(a) all significant deficiencies and material weaknesses in the design or operation of internal control over financial reporting which are reasonably\nlikely to adversely affect the registrant's ability to record, process, summarize and report financial information; and\n(b) any fraud, whether or not material, that involves management or other employees who have a significant role in the registrant's internal\ncontrol over financial reporting.\nDate: February 21, 2024\n \n \n/s/ JEN-HSUN HUANG\nJen-Hsun Huang \nPresident and Chief Executive Officer..."
3,NVIDIA_Report_2024.pdf,87,"EXHIBIT 31.1\nCERTIFICATION\n \nI, Jen-Hsun Huang, certify that:\n \n1. I have reviewed this Annual Report on Form 10-K of NVIDIA Corporation;\n2. Based on my knowledge, this report does not contain any untrue statement of a material fact or omit to state a material fact necessary to\nmake the statements made, in light of the circumstances under which such statements were made, not misleading with respect to the period\ncovered by this report;\n \n3. Based on my knowledge, the financial statements, and other financial information included in this report, fairly present in all material respects\nthe financial condition, results of operations and cash flows of the registrant as of, and for, the periods presented in this report;\n \n4. The registrant's other certifying officer(s) and I are responsible for establishing and maintaining disclosure controls and procedures (as..."
4,NVIDIA_Report_2024.pdf,83,"Signatures\nPursuant to the requirements of Section 13 or 15(d) of the Securities Exchange Act of 1934, the Registrant has duly caused this report to be signed on its behalfby the undersigned, thereunto duly authorized, on February 21, 2024.\nNVIDIA Corporation\nBy: /s/ Jen-Hsun Huang \n Jen-Hsun Huang\n President and Chief Executive Officer\n

In [18]:
result["answer"]

'The CEO of the Company is Jen-Hsun Huang.'

In [19]:
len(result["contexts"])

5

## RAG Evaluation Dataset 

In [21]:
# Import the LangSmith client
from langsmith import Client

# Initialize the LangSmith client
ls_client = Client()

# Read an existing dataset by name
dataset_name = "RAG_Eval_QA"
dataset = ls_client.read_dataset(dataset_name=dataset_name)

# List examples in the dataset
examples = ls_client.list_examples(dataset_name=dataset_name)

# Iterate through examples
for example in examples:
    print(example.inputs)  # Print input data
    print(example.outputs)  # Print output data


{'question': "What was NVIDIA's revenue in 2024 and 2023? Which year saw higher revenues and by how much?"}
{'answer': "Revenue in 2024 was $60,922 million ($60.9 billion) , revenue in 2023 was $26,974 million ($27.0 billion). NVIDIA's revenue increased by $33,948 million (or $33.95 billion depending on the rounding) in 2024 compared to 2023."}
{'question': 'Between January 29, 2023, and January 28, 2024, NVIDIA’s total assets increased by approximately $24.5 billion. Based on the balance sheet, which two line items contributed the most to this increase, and what does this suggest about the company’s capital deployment focus?'}
{'answer': 'Between January 29, 2023, and January 28, 2024, NVIDIA’s total assets increased by approximately $24.5 billion, rising from $41.2 billion to $65.7 billion. Based on the balance sheet, the two line items that contributed the most to this increase were marketable securities, which increased from $9.9 billion to $18.7 billion, and cash and cash equivale

In [22]:
from langchain_core.rate_limiters import InMemoryRateLimiter

rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.1,  # Slow down requests
    check_every_n_seconds=0.1,
    max_bucket_size=10
)

# Apply to your model

model = ChatOpenAI(model="gpt-4o", temperature=0, rate_limiter=rate_limiter)

/var/folders/8_/66zkglc51d973l0mk64lcrkc0000gn/T/ipykernel_4588/2168577922.py:3: LangChainBetaWarning: Introduced in 0.2.24. API subject to change.
  rate_limiter = InMemoryRateLimiter(


## RAG Evaluators Propositions

### Type 1: Reference Answer

First, lets consider the case in which we want to compare our RAG chain answer to a reference answer.

#### Eval flow

We simply use an LLM-as-judge with an easily customized grader prompt: 

In [23]:
# RAG chain
def predict_rag_answer(example: dict):
    """Use this for answer evaluation"""
    response =  query_rag(example["question"])
    return {"answer": response["answer"]}

In [24]:
def predict_rag_answer_with_context(example: dict):
    """Use this for evaluation of retrieved documents and hallucinations"""
    response = query_rag(example["question"])
    return {"answer": response["answer"], "contexts": response["contexts"]}

### RAG Answer VS Reference prompt

In [25]:
# set the LANGSMITH_API_KEY environment variable (create key in settings)
from langchain import hub
grade_prompt_answer_accuracy = hub.pull("answer_vs_reference")

In [26]:
def answer_evaluator(run, example) -> dict:
    """
    A simple evaluator for RAG answer accuracy
    """
    
    # Get summary
    input_question = example.inputs["question"]
    reference = example.outputs["answer"]
    prediction = run.outputs["answer"]

    # LLM grader
    # llm = ChatOpenAI(model="gpt-4o", temperature=0)
    
    llm = model
 
    # Structured prompt
    
    answer_grader = grade_prompt_answer_accuracy | llm

    # Get score
    score = answer_grader.invoke({"question": input_question,
                                  "correct_answer": reference,
                                  "student_answer": prediction})
    score = score["Score"]

    return {"key": "answer_score", "score": score}

In [41]:
from langsmith.evaluation import evaluate

dataset_name = "RAG_test_NVIDIA"
experiment_results = evaluate(
    predict_rag_answer,
    data=dataset_name,
    evaluators=[answer_evaluator],
    experiment_prefix="Parent-Child",
    metadata={"variant": "NVIDIA context, gpt-4o-mini"},
)

View the evaluation results for experiment: 'Parent-Child-c6ea9cd8' at:
https://smith.langchain.com/o/f09c54fc-44b8-4ffe-9055-bc40601f9137/datasets/7fad5e82-7b2d-438e-90e6-1e232f388896/compare?selectedSessions=cb9fa54e-ed71-4afa-97d8-78b69bbb9944




0it [00:00, ?it/s]

,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,87,"Jen-Hsun Huang, CEO of NVIDIA, certifies the accuracy of the company's Annual Report on Form 10-K, affirming that it contains no misleading statements and fairly represents the financial condition. He outlines management's responsibilities for disclosure controls and internal financial reporting, and discloses any significant deficiencies or fraud. The certification is dated February 21, 2024...."
1,NVIDIA_Report_2024.pdf,83,"On February 21, 2024, NVIDIA Corporation submitted a report, signed by CEO Jen-Hsun Huang, in accordance with the Securities Exchange Act of 1934. The report includes a Power of Attorney designating Huang and Colette M.Kress as attorneys-in-fact to sign amendments and file related documents with the SEC...."
2,NVIDIA_Report_2024.pdf,89,"Jen-Hsun Huang, CEO of NVIDIA, certifies that the company's Annual Report on Form 10-K for the year ending January 28, 2024, complies with the Securities Exchange Act and accurately reflects its financial status. Dated February 21, 2024, the certification will be retained by NVIDIA and provided to the SEC upon request, but is not filed with the SEC or included in other filings...."


,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,87,"EXHIBIT 31.1\nCERTIFICATION\n \nI, Jen-Hsun Huang, certify that:\n \n1. I have reviewed this Annual Report on Form 10-K of NVIDIA Corporation;\n2. Based on my knowledge, this report does not contain any untrue statement of a material fact or omit to state a material fact necessary to\nmake the statements made, in light of the circumstances under which such statements were made, not misleading with respect to the period\ncovered by this report;\n \n3. Based on my knowledge, the financial statements, and other financial information included in this report, fairly present in all material respects\nthe financial condition, results of operations and cash flows of the registrant as of, and for, the periods presented in this report;\n \n4. The registrant's other certifying officer(s) and I are responsible for establishing and maintaining disclosure controls and procedures (as..."
1,NVIDIA_Report_2024.pdf,83,"Signatures\nPursuant to the requirements of Section 13 or 15(d) of the Securities Exchange Act of 1934, the Registrant has duly caused this report to be signed on its behalfby the undersigned, thereunto duly authorized, on February 21, 2024.\nNVIDIA Corporation\nBy: /s/ Jen-Hsun Huang \n Jen-Hsun Huang\n President and Chief Executive Officer\nPower of Attorney\nKNOW ALL PERSONS BY THESE PRESENTS, that each person whose signature appears below constitutes and appoints Jen-Hsun Huang and Colette M.Kress, and each or any one of them, his true and lawful attorney-in-fact and agent, with full power of substitution and resubstitution, for him and in his name,..."
2,NVIDIA_Report_2024.pdf,89,"EXHIBIT 32.1\n \nCERTIFICATION \n \nPursuant to the requirement set forth in Rule 13a-14(b) of the Securities Exchange Act of 1934, as amended (the “Exchange Act”), and\nSection 1350 of Chapter 63 of Title 18 of the United States Code (18 U.S.C. § 1350), Jen-Hsun Huang, the President and Chief Executive\nOfficer of NVIDIA Corporation (the “Company”), hereby certifies that, to the best of his knowledge:\n1. The Company's Annual Report on Form 10-K for the year ended January 28, 2024, to which this Certification is attached as Exhibit 32.1 (the\n“Periodic Report”), fully complies with the requirements of Section 13(a) or Section 15(d) of the Exchange Act; and\n2. The information contained in the Periodic Report fairly presents, in all material respects, the financial condition of the Company at the end of\nthe period covered by the Periodic Report and results of operations of the Company for the period covered by the Periodic Report.\nDate: February 21, 2024\n \n \n/s/ JEN-HSUN HUANG..."
3,NVIDIA_Report_2024.pdf,89,"the period covered by the Periodic Report and results of operations of the Company for the period covered by the Periodic Report.\nDate: February 21, 2024\n \n \n/s/ JEN-HSUN HUANG\nJen-Hsun Huang \nPresident and Chief Executive Officer\n \n \nA signed original of this written statement required by Section 906 of 18 U.S.C. § 1350 has been provided to NVIDIA Corporation and will be\nretained by NVIDIA Corporation and furnished to the Securities and Exchange Commission or its staff upon request.\n \nThis certification accompanies the Form 10-K to which it relates, is not deemed filed with the Securities and Exchange Commission and is\nnot to be incorporated by reference into any filing of the Company under the Securities Act of 1933, as amended, or the Exchange Act (whether\nmade before or after the date of the Form 10-K), irrespective of any general incorporation language contained in such filing...."


,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,4,"NVIDIA offers advanced computing solutions for over 3,500 applications in fields like molecular dynamics and genomics, powering over 75% of the world's top supercomputers. The company operates in two main segments: Compute & Networking, focusing on data center and AI solutions, and Graphics, which includes gaming GPUs. Its key markets are Data Center, Gaming, Professional Visualization, and Automotive, serving a diverse clientele, including cloud providers and public sector entities...."
1,NVIDIA_Report_2024.pdf,3,"NVIDIA is a leader in accelerated computing, offering GPUs, software libraries, and APIs that enhance performance for AI, data analytics, and 3D graphics. Their architecture supports large-scale data centers and has driven advancements in AI through innovations like the CUDA programming model and Tensor Core GPUs. With over $45.3 billion invested in R&D, NVIDIA provides solutions across various industries, including healthcare and finance, and their platforms are widely used by cloud service providers and enterprises for generative AI and workflow optimization...."
2,NVIDIA_Report_2024.pdf,5,"NVIDIA's accelerated computing platform leverages powerful GPUs for parallel workloads, particularly in neural network training and inferencing, and has expanded to include DPUs and CPUs. Key software offerings include CUDA, NVIDIA AI Enterprise, and NVIDIA vGPU. In fiscal year 2024, NVIDIA launched DGX Cloud, an AI-training-as-a-service platform. In gaming, NVIDIA enhances experiences with RTX technology, including ray tracing and DLSS. The company also supports professional visualization through partnerships and its Omniverse platform. In the automotive sector, NVIDIA offers AI-driven solutions for automated driving and in-vehicle computing under the DRIVE Hyperion brand...."


,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,4,"infotainment systems; and Omniverse Enterprise software for building and operating metaverse and 3D internet applications.\nOur Markets\nWe specialize in markets where our computing platforms can provide tremendous acceleration for applications. These platforms incorporate processors,interconnects, software, algorithms, systems, and services to deliver unique value. Our platforms address four large markets where our expertise is critical: DataCenter, Gaming, Professional Visualization, and Automotive.\nData Center\nThe NVIDIA Data Center platform is focused on accelerating the most compute-intensive workloads, such as AI, data analytics, graphics and scientific..."
1,NVIDIA_Report_2024.pdf,4,"broader set of PC users to choose NVIDIA GPUs for running generative AI applications locally on their PC, which is critical for privacy, latency, and cost-sensitiveAI applications.\nProfessional artists, architects and designers use NVIDIA partner products accelerated with our GPUs and software platform for a range of creative and designuse cases, such as creating visual effects in movies or designing buildings and products. In addition, generative AI is expanding the market for our workstation-class GPUs, as more enterprise customers develop and deploy AI applications with their data on-premises.\nHeadquartered in Santa Clara, California, NVIDIA was incorporated in California in April 1993 and reincorporated in Delaware in April 1998.\nOur Businesses\nWe report our business results in two segments...."
2,NVIDIA_Report_2024.pdf,4,"Our Businesses\nWe report our business results in two segments.\nThe Compute & Networking segment is comprised of our Data Center accelerated computing platforms and end-to-end networking platforms including Quantumfor InfiniBand and Spectrum for Ethernet; our NVIDIA DRIVE automated-driving platform and automotive development agreements; Jetson robotics and other\nembedded platforms; NVIDIA AI Enterprise and other software; and DGX Cloud software and services.\nThe Graphics segment includes GeForce GPUs for gaming and PCs, the GeForce NOW game streaming service and related infrastructure; Quadro/NVIDIARTX GPUs for enterprise workstation graphics; virtual GPU, or vGPU, software for cloud-based visual and virtual computing; automotive platforms for\ninfotainment systems; and Omniverse Enterprise software for building and operating metaverse and 3D internet applications.\nOur Markets..."
3,NVIDIA_Report_2024.pdf,4,"Table of Contents\nResearchers and developers use our computing solutions to accelerate a wide range of important applications, from simulating molecular dynamics to climate\nforecasting. With support for more than 3,500 applications, NVIDIA computing enables some of the most promising areas of discovery, from climate prediction tomaterials science and from wind tunnel simulation to genomics. Including GPUs and networking, NVIDIA powers over 75% of the supercomputers on the global\nTOP500 list, including 24 of the top 30 systems on the Green500 list.\nGamers choose NVIDIA GPUs to enjoy immersive, increasingly cinematic virtual worlds. In addition to serving the growing number of gamers, the market for PCGPUs is expanding because of the burgeoning population of live streamers, broadcasters, artists, and creators. With the advent of generative AI, we expect a..."
4,NVIDIA_Report_2024.pdf,4,"Data Center\nThe NVIDIA Data Center platform is focused on accelerating the most compute-intensive workloads, such as AI, data analytics, graphics and scientific\ncomputing, delivering significantly better performance and power efficiency relative to conventional CPU-only approaches. It is deployed in cloud, hyperscale,on-premises and edge data centers. The platform consists of compute and networking offerings typically delivered to customers as systems, subsystems, ormodules, along with software and services.\nOur compute offerings include supercomp

,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,78,"NVIDIA Corporation's financial statements for the fiscal year ending January 28, 2024, show total revenue of $60.9 billion, up from $27.0 billion in 2023. The Data Center segment led with $47.5 billion, while the Gaming segment contributed $10.4 billion. U.S. revenue reached $27.0 billion, with international sales making up 56% of total revenue, down from 69% in 2023. The report also notes $4.9 billion in reconciling items, including stock-based compensation and acquisition-related costs...."
1,NVIDIA_Report_2024.pdf,35,"In fiscal year 2024, NVIDIA's revenue reached $60.9 billion, a 126% increase from the previous year. Data Center revenue soared 217% to $47.5 billion, fueled by demand for AI and cloud services. Gaming revenue grew 15% to $10.4 billion, while Professional Visualization and Automotive revenues increased by 1% and 21%, respectively. Gross margin improved to 72.7%, and net income surged 581% to $29.8 billion, despite a slight rise in operating expenses due to employee growth and compensation...."
2,NVIDIA_Report_2024.pdf,49,"For the year ending January 28, 2024, NVIDIA Corporation reported a revenue of $60.92 billion, up from $26.97 billion in 2023. The cost of revenue was $16.62 billion, resulting in a gross profit of $44.30 billion. Operating expenses increased to $11.33 billion, yielding an operating income of $32.97 billion. After taxes, the net income was $29.76 billion, equating to a diluted net income per share of $11.93...."


,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,78,"The following table summarizes information pertaining to our revenue by each of the specialized markets we serve:\n Year Ended\n Jan 28, 2024 Jan 29, 2023 Jan 30, 2022\nRevenue: (In millions)\nData Center $ 47,525 $ 15,005 $ 10,613 \nGaming 10,447 9,067 12,462 \nProfessional Visualization 1,553 1,544 2,111 \nAutomotive 1,091 903 566 \nOEM and Other 306 455 1,162 \nTotal revenue $ 60,922 $ 26,974 $ 26,914 \n79..."
1,NVIDIA_Report_2024.pdf,78,"Table of Contents\nNVIDIA Corporation and Subsidiaries\nNotes to the Consolidated Financial Statements(Continued)\nYear Ended\nJan 28, 2024 Jan 29, 2023 Jan 30, 2022\n(In millions)\nReconciling items included in ""All Other"" category:\nStock-based compensation expense $ (3,549)$ (2,710)$ (2,004)\nUnallocated cost of revenue and operating expenses (728) (595) (399)\nAcquisition-related and other costs (583) (674) (636)\nIP-related and legal settlement costs (40) (23) (10)\nRestructuring costs and other — (54) — \nAcquisition termination cost — (1,353) — \nOther 10 (2) — \nTotal $ (4,890)$ (5,411) $ (3,049)\nRevenue by geographic areas is designated based upon the billing location of the customer. End customer location may be different than our customer’s billing\nlocation. Revenue by geographic areas was as follows: \n Year Ended\n Jan 28, 2024 Jan 29, 2023 Jan 30, 2022\nRevenue: (In millions)\nUnited States $ 26,966 $ 8,292 $ 4,349 \nTaiwan 13,405 6,986 8,544 \nChina (including Hong Kong) 10,306 5,785 7,111..."
2,NVIDIA_Report_2024.pdf,78,"Year Ended\n Jan 28, 2024 Jan 29, 2023 Jan 30, 2022\nRevenue: (In millions)\nUnited States $ 26,966 $ 8,292 $ 4,349 \nTaiwan 13,405 6,986 8,544 \nChina (including Hong Kong) 10,306 5,785 7,111 \nOther countries 10,245 5,911 6,910 \nTotal revenue $ 60,922 $ 26,974 $ 26,914 \nRevenue from sales to customers outside of the United States accounted for 56%, 69%, and 84% of total revenue for fiscal years 2024, 2023, and 2022,\nrespectively. The increase in revenue to the United States for fiscal year 2024 was primarily due to higher U.S.-based Compute & Networking segment demand.\nSales to one customer represented 13% of total revenue for fiscal year 2024, which was attributable to the Compute & Networking segment. No customerrepresented 10% or more of total revenue for fiscal years 2023 and 2022.\nThe following table summarizes information pertaining to our revenue by each of the specialized markets we serve:\n Year Ended\n Jan 28, 2024 Jan 29, 2023 Jan 30, 2022\nRevenue: (In millions)..."
3,NVIDIA_Report_2024.pdf,35,"Revenue for fiscal year 2024 was $60.9 billion, up 126% from a year ago.\nData Center revenue for fiscal year 2024 was up 217%. Strong demand was driven by enterprise software and consumer internet applications, and multipleindustry verticals including automotive, financial services, and healthcare. Customers across industry verticals access NVIDIA AI infrastructure both through the\ncloud and on-premises. Data Center compute revenue was up 244% in the fiscal year. Networking revenue was up 133% in the fiscal year.\nGaming revenue for fiscal year 2024 was up 15%. The increase reflects higher sell-in to partners following the normalization of channel inventory levels and\ngrowing demand.\nProfessional Visualization revenue for fiscal year 2024 was up 1%.\nAutomotive revenue for the fiscal year 2024 was up 21%. The increase primarily reflected growth in self-driving platforms...."
4,NVIDIA_Report_2024.pdf,35,"Table of Contents\nFiscal Year 2024 Summary\n Year Ended\n Jan 28, 2024 Jan 29, 2023 Change\n($ in millions, except per share data)\nRevenue $ 60,922 $ 26,974 Up 126%\nGross margin 72.7 % 56.9 % Up 15.8 pts\nOperating expenses $ 11,329 $ 11,132 Up 2%\nOperating income $ 32,972 $ 4,224 Up 681%\nNet income $ 29,760 $ 4,368 Up 581%\nNet income per diluted share $ 11.93 $ 1.74 Up 586%\nWe specialize in markets where our computing platforms can provide tremendous acceleration

### Type 2: Answer Hallucination

#### Eval flow

We simply use an LLM-as-judge with an easily customized grader prompt: 

https://smith.langchain.com/hub/langchain-ai/rag-answer-hallucination

In [27]:
# set the LANGSMITH_API_KEY environment variable (create key in settings)
from langchain import hub
grade_prompt_hallucinations = hub.pull("hallucination")

In [28]:
# Prompt 

# grade_prompt_hallucinations = hub.pull("langchain-ai/rag-answer-hallucination")

def answer_hallucination_evaluator(run, example) -> dict:
    """
    A simple evaluator for generation hallucination
    """
    
    # RAG inputs
    input_question = example.inputs["question"]
    contexts = run.outputs["contexts"]
        
    # RAG answer 
    prediction = run.outputs["answer"]

    # LLM grader
    # llm = ChatOpenAI(model="gpt-4o", temperature=0)
    
    llm = model

    # Structured prompt
    answer_grader = grade_prompt_hallucinations | llm

    # Get score
    score = answer_grader.invoke({"documents": contexts,
                                  "student_answer": prediction})
    score = score["Score"]

    return {"key": "answer_hallucination", "score": score}

### Type 3: Document Relevance to Question

#### Eval flow

We simply use an LLM-as-judge with an easily customized grader prompt: 

https://smith.langchain.com/hub/langchain-ai/rag-document-relevance

In [30]:
# set the LANGSMITH_API_KEY environment variable (create key in settings)
from langchain import hub
grade_prompt_doc_relevance = hub.pull("doc_question_relevance")

In [31]:
# Grade prompt 
# grade_prompt_doc_relevance = hub.pull("langchain-ai/rag-document-relevance")

def docs_relevance_evaluator(run, example) -> dict:
    """
    A simple evaluator for document relevance
    """
    
    # RAG inputs
    input_question = example.inputs["question"]
    contexts = run.outputs["contexts"]
        
    # RAG answer 
    prediction = run.outputs["answer"]

    # LLM grader
    # llm = ChatOpenAI(model="gpt-4o", temperature=0)
    
    llm = model

    # Structured prompt
    answer_grader = grade_prompt_doc_relevance | llm

    # Get score
    score = answer_grader.invoke({"question":input_question,
                                  "documents":contexts})
    score = score["Score"]

    return {"key": "document_relevance", "score": score}

In [47]:
dataset_name = "RAG_test_NVIDIA"
experiment_results = evaluate(
    predict_rag_answer_with_context,
    data=dataset_name,
    evaluators=[docs_relevance_evaluator],
    experiment_prefix="Parent-Child",
    metadata={"variant": "NVIDIA context, gpt-4o-mini"},
)

View the evaluation results for experiment: 'Parent-Child-3491dfef' at:
https://smith.langchain.com/o/f09c54fc-44b8-4ffe-9055-bc40601f9137/datasets/7fad5e82-7b2d-438e-90e6-1e232f388896/compare?selectedSessions=a4a4f693-9ba0-4bfa-91c2-0734c3b32f28




0it [00:00, ?it/s]

,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,87,"Jen-Hsun Huang, CEO of NVIDIA, certifies the accuracy of the company's Annual Report on Form 10-K, affirming that it contains no misleading statements and fairly represents the financial condition. He outlines management's responsibilities for disclosure controls and internal financial reporting, and discloses any significant deficiencies or fraud. The certification is dated February 21, 2024...."
1,NVIDIA_Report_2024.pdf,83,"On February 21, 2024, NVIDIA Corporation submitted a report, signed by CEO Jen-Hsun Huang, in accordance with the Securities Exchange Act of 1934. The report includes a Power of Attorney designating Huang and Colette M.Kress as attorneys-in-fact to sign amendments and file related documents with the SEC...."
2,NVIDIA_Report_2024.pdf,89,"Jen-Hsun Huang, CEO of NVIDIA, certifies that the company's Annual Report on Form 10-K for the year ending January 28, 2024, complies with the Securities Exchange Act and accurately reflects its financial status. Dated February 21, 2024, the certification will be retained by NVIDIA and provided to the SEC upon request, but is not filed with the SEC or included in other filings...."


,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,87,"EXHIBIT 31.1\nCERTIFICATION\n \nI, Jen-Hsun Huang, certify that:\n \n1. I have reviewed this Annual Report on Form 10-K of NVIDIA Corporation;\n2. Based on my knowledge, this report does not contain any untrue statement of a material fact or omit to state a material fact necessary to\nmake the statements made, in light of the circumstances under which such statements were made, not misleading with respect to the period\ncovered by this report;\n \n3. Based on my knowledge, the financial statements, and other financial information included in this report, fairly present in all material respects\nthe financial condition, results of operations and cash flows of the registrant as of, and for, the periods presented in this report;\n \n4. The registrant's other certifying officer(s) and I are responsible for establishing and maintaining disclosure controls and procedures (as..."
1,NVIDIA_Report_2024.pdf,83,"Signatures\nPursuant to the requirements of Section 13 or 15(d) of the Securities Exchange Act of 1934, the Registrant has duly caused this report to be signed on its behalfby the undersigned, thereunto duly authorized, on February 21, 2024.\nNVIDIA Corporation\nBy: /s/ Jen-Hsun Huang \n Jen-Hsun Huang\n President and Chief Executive Officer\nPower of Attorney\nKNOW ALL PERSONS BY THESE PRESENTS, that each person whose signature appears below constitutes and appoints Jen-Hsun Huang and Colette M.Kress, and each or any one of them, his true and lawful attorney-in-fact and agent, with full power of substitution and resubstitution, for him and in his name,..."
2,NVIDIA_Report_2024.pdf,89,"EXHIBIT 32.1\n \nCERTIFICATION \n \nPursuant to the requirement set forth in Rule 13a-14(b) of the Securities Exchange Act of 1934, as amended (the “Exchange Act”), and\nSection 1350 of Chapter 63 of Title 18 of the United States Code (18 U.S.C. § 1350), Jen-Hsun Huang, the President and Chief Executive\nOfficer of NVIDIA Corporation (the “Company”), hereby certifies that, to the best of his knowledge:\n1. The Company's Annual Report on Form 10-K for the year ended January 28, 2024, to which this Certification is attached as Exhibit 32.1 (the\n“Periodic Report”), fully complies with the requirements of Section 13(a) or Section 15(d) of the Exchange Act; and\n2. The information contained in the Periodic Report fairly presents, in all material respects, the financial condition of the Company at the end of\nthe period covered by the Periodic Report and results of operations of the Company for the period covered by the Periodic Report.\nDate: February 21, 2024\n \n \n/s/ JEN-HSUN HUANG..."
3,NVIDIA_Report_2024.pdf,89,"the period covered by the Periodic Report and results of operations of the Company for the period covered by the Periodic Report.\nDate: February 21, 2024\n \n \n/s/ JEN-HSUN HUANG\nJen-Hsun Huang \nPresident and Chief Executive Officer\n \n \nA signed original of this written statement required by Section 906 of 18 U.S.C. § 1350 has been provided to NVIDIA Corporation and will be\nretained by NVIDIA Corporation and furnished to the Securities and Exchange Commission or its staff upon request.\n \nThis certification accompanies the Form 10-K to which it relates, is not deemed filed with the Securities and Exchange Commission and is\nnot to be incorporated by reference into any filing of the Company under the Securities Act of 1933, as amended, or the Exchange Act (whether\nmade before or after the date of the Form 10-K), irrespective of any general incorporation language contained in such filing...."


,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,78,"NVIDIA Corporation's financial statements for the fiscal year ending January 28, 2024, show total revenue of $60.9 billion, up from $27.0 billion in 2023. The Data Center segment led with $47.5 billion, while the Gaming segment contributed $10.4 billion. U.S. revenue reached $27.0 billion, with international sales making up 56% of total revenue, down from 69% in 2023. The report also notes $4.9 billion in reconciling items, including stock-based compensation and acquisition-related costs...."
1,NVIDIA_Report_2024.pdf,35,"In fiscal year 2024, NVIDIA's revenue reached $60.9 billion, a 126% increase from the previous year. Data Center revenue soared 217% to $47.5 billion, fueled by demand for AI and cloud services. Gaming revenue grew 15% to $10.4 billion, while Professional Visualization and Automotive revenues increased by 1% and 21%, respectively. Gross margin improved to 72.7%, and net income surged 581% to $29.8 billion, despite a slight rise in operating expenses due to employee growth and compensation...."
2,NVIDIA_Report_2024.pdf,49,"For the year ending January 28, 2024, NVIDIA Corporation reported a revenue of $60.92 billion, up from $26.97 billion in 2023. The cost of revenue was $16.62 billion, resulting in a gross profit of $44.30 billion. Operating expenses increased to $11.33 billion, yielding an operating income of $32.97 billion. After taxes, the net income was $29.76 billion, equating to a diluted net income per share of $11.93...."


,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,78,"The following table summarizes information pertaining to our revenue by each of the specialized markets we serve:\n Year Ended\n Jan 28, 2024 Jan 29, 2023 Jan 30, 2022\nRevenue: (In millions)\nData Center $ 47,525 $ 15,005 $ 10,613 \nGaming 10,447 9,067 12,462 \nProfessional Visualization 1,553 1,544 2,111 \nAutomotive 1,091 903 566 \nOEM and Other 306 455 1,162 \nTotal revenue $ 60,922 $ 26,974 $ 26,914 \n79..."
1,NVIDIA_Report_2024.pdf,78,"Table of Contents\nNVIDIA Corporation and Subsidiaries\nNotes to the Consolidated Financial Statements(Continued)\nYear Ended\nJan 28, 2024 Jan 29, 2023 Jan 30, 2022\n(In millions)\nReconciling items included in ""All Other"" category:\nStock-based compensation expense $ (3,549)$ (2,710)$ (2,004)\nUnallocated cost of revenue and operating expenses (728) (595) (399)\nAcquisition-related and other costs (583) (674) (636)\nIP-related and legal settlement costs (40) (23) (10)\nRestructuring costs and other — (54) — \nAcquisition termination cost — (1,353) — \nOther 10 (2) — \nTotal $ (4,890)$ (5,411) $ (3,049)\nRevenue by geographic areas is designated based upon the billing location of the customer. End customer location may be different than our customer’s billing\nlocation. Revenue by geographic areas was as follows: \n Year Ended\n Jan 28, 2024 Jan 29, 2023 Jan 30, 2022\nRevenue: (In millions)\nUnited States $ 26,966 $ 8,292 $ 4,349 \nTaiwan 13,405 6,986 8,544 \nChina (including Hong Kong) 10,306 5,785 7,111..."
2,NVIDIA_Report_2024.pdf,78,"Year Ended\n Jan 28, 2024 Jan 29, 2023 Jan 30, 2022\nRevenue: (In millions)\nUnited States $ 26,966 $ 8,292 $ 4,349 \nTaiwan 13,405 6,986 8,544 \nChina (including Hong Kong) 10,306 5,785 7,111 \nOther countries 10,245 5,911 6,910 \nTotal revenue $ 60,922 $ 26,974 $ 26,914 \nRevenue from sales to customers outside of the United States accounted for 56%, 69%, and 84% of total revenue for fiscal years 2024, 2023, and 2022,\nrespectively. The increase in revenue to the United States for fiscal year 2024 was primarily due to higher U.S.-based Compute & Networking segment demand.\nSales to one customer represented 13% of total revenue for fiscal year 2024, which was attributable to the Compute & Networking segment. No customerrepresented 10% or more of total revenue for fiscal years 2023 and 2022.\nThe following table summarizes information pertaining to our revenue by each of the specialized markets we serve:\n Year Ended\n Jan 28, 2024 Jan 29, 2023 Jan 30, 2022\nRevenue: (In millions)..."
3,NVIDIA_Report_2024.pdf,35,"Revenue for fiscal year 2024 was $60.9 billion, up 126% from a year ago.\nData Center revenue for fiscal year 2024 was up 217%. Strong demand was driven by enterprise software and consumer internet applications, and multipleindustry verticals including automotive, financial services, and healthcare. Customers across industry verticals access NVIDIA AI infrastructure both through the\ncloud and on-premises. Data Center compute revenue was up 244% in the fiscal year. Networking revenue was up 133% in the fiscal year.\nGaming revenue for fiscal year 2024 was up 15%. The increase reflects higher sell-in to partners following the normalization of channel inventory levels and\ngrowing demand.\nProfessional Visualization revenue for fiscal year 2024 was up 1%.\nAutomotive revenue for the fiscal year 2024 was up 21%. The increase primarily reflected growth in self-driving platforms...."
4,NVIDIA_Report_2024.pdf,35,"Table of Contents\nFiscal Year 2024 Summary\n Year Ended\n Jan 28, 2024 Jan 29, 2023 Change\n($ in millions, except per share data)\nRevenue $ 60,922 $ 26,974 Up 126%\nGross margin 72.7 % 56.9 % Up 15.8 pts\nOperating expenses $ 11,329 $ 11,132 Up 2%\nOperating income $ 32,972 $ 4,224 Up 681%\nNet income $ 29,760 $ 4,368 Up 581%\nNet income per diluted share $ 11.93 $ 1.74 Up 586%\nWe specialize in markets where our computing platforms can provide tremendous acceleration

,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,4,"NVIDIA offers advanced computing solutions for over 3,500 applications in fields like molecular dynamics and genomics, powering over 75% of the world's top supercomputers. The company operates in two main segments: Compute & Networking, focusing on data center and AI solutions, and Graphics, which includes gaming GPUs. Its key markets are Data Center, Gaming, Professional Visualization, and Automotive, serving a diverse clientele, including cloud providers and public sector entities...."
1,NVIDIA_Report_2024.pdf,3,"NVIDIA is a leader in accelerated computing, offering GPUs, software libraries, and APIs that enhance performance for AI, data analytics, and 3D graphics. Their architecture supports large-scale data centers and has driven advancements in AI through innovations like the CUDA programming model and Tensor Core GPUs. With over $45.3 billion invested in R&D, NVIDIA provides solutions across various industries, including healthcare and finance, and their platforms are widely used by cloud service providers and enterprises for generative AI and workflow optimization...."
2,NVIDIA_Report_2024.pdf,5,"NVIDIA's accelerated computing platform leverages powerful GPUs for parallel workloads, particularly in neural network training and inferencing, and has expanded to include DPUs and CPUs. Key software offerings include CUDA, NVIDIA AI Enterprise, and NVIDIA vGPU. In fiscal year 2024, NVIDIA launched DGX Cloud, an AI-training-as-a-service platform. In gaming, NVIDIA enhances experiences with RTX technology, including ray tracing and DLSS. The company also supports professional visualization through partnerships and its Omniverse platform. In the automotive sector, NVIDIA offers AI-driven solutions for automated driving and in-vehicle computing under the DRIVE Hyperion brand...."


,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,4,"infotainment systems; and Omniverse Enterprise software for building and operating metaverse and 3D internet applications.\nOur Markets\nWe specialize in markets where our computing platforms can provide tremendous acceleration for applications. These platforms incorporate processors,interconnects, software, algorithms, systems, and services to deliver unique value. Our platforms address four large markets where our expertise is critical: DataCenter, Gaming, Professional Visualization, and Automotive.\nData Center\nThe NVIDIA Data Center platform is focused on accelerating the most compute-intensive workloads, such as AI, data analytics, graphics and scientific..."
1,NVIDIA_Report_2024.pdf,4,"broader set of PC users to choose NVIDIA GPUs for running generative AI applications locally on their PC, which is critical for privacy, latency, and cost-sensitiveAI applications.\nProfessional artists, architects and designers use NVIDIA partner products accelerated with our GPUs and software platform for a range of creative and designuse cases, such as creating visual effects in movies or designing buildings and products. In addition, generative AI is expanding the market for our workstation-class GPUs, as more enterprise customers develop and deploy AI applications with their data on-premises.\nHeadquartered in Santa Clara, California, NVIDIA was incorporated in California in April 1993 and reincorporated in Delaware in April 1998.\nOur Businesses\nWe report our business results in two segments...."
2,NVIDIA_Report_2024.pdf,4,"Our Businesses\nWe report our business results in two segments.\nThe Compute & Networking segment is comprised of our Data Center accelerated computing platforms and end-to-end networking platforms including Quantumfor InfiniBand and Spectrum for Ethernet; our NVIDIA DRIVE automated-driving platform and automotive development agreements; Jetson robotics and other\nembedded platforms; NVIDIA AI Enterprise and other software; and DGX Cloud software and services.\nThe Graphics segment includes GeForce GPUs for gaming and PCs, the GeForce NOW game streaming service and related infrastructure; Quadro/NVIDIARTX GPUs for enterprise workstation graphics; virtual GPU, or vGPU, software for cloud-based visual and virtual computing; automotive platforms for\ninfotainment systems; and Omniverse Enterprise software for building and operating metaverse and 3D internet applications.\nOur Markets..."
3,NVIDIA_Report_2024.pdf,4,"Table of Contents\nResearchers and developers use our computing solutions to accelerate a wide range of important applications, from simulating molecular dynamics to climate\nforecasting. With support for more than 3,500 applications, NVIDIA computing enables some of the most promising areas of discovery, from climate prediction tomaterials science and from wind tunnel simulation to genomics. Including GPUs and networking, NVIDIA powers over 75% of the supercomputers on the global\nTOP500 list, including 24 of the top 30 systems on the Green500 list.\nGamers choose NVIDIA GPUs to enjoy immersive, increasingly cinematic virtual worlds. In addition to serving the growing number of gamers, the market for PCGPUs is expanding because of the burgeoning population of live streamers, broadcasters, artists, and creators. With the advent of generative AI, we expect a..."
4,NVIDIA_Report_2024.pdf,4,"Data Center\nThe NVIDIA Data Center platform is focused on accelerating the most compute-intensive workloads, such as AI, data analytics, graphics and scientific\ncomputing, delivering significantly better performance and power efficiency relative to conventional CPU-only approaches. It is deployed in cloud, hyperscale,on-premises and edge data centers. The platform consists of compute and networking offerings typically delivered to customers as systems, subsystems, ormodules, along with software and services.\nOur compute offerings include supercomp

# Complete Evaluation

In [32]:
from langsmith.evaluation import evaluate

dataset_name = "RAG_Eval_QA"
experiment_results = evaluate(
    predict_rag_answer_with_context,
    data=dataset_name,
    evaluators=[answer_evaluator, answer_hallucination_evaluator, docs_relevance_evaluator],
    experiment_prefix="Parent_Child",
    metadata={"variant": "NVIDIA context, gpt-4o-mini"},
)

View the evaluation results for experiment: 'Parent_Child-2788e687' at:
https://smith.langchain.com/o/f09c54fc-44b8-4ffe-9055-bc40601f9137/datasets/176f75c1-604f-435b-b0c7-9eda1bdef775/compare?selectedSessions=f2939241-e536-45b2-974e-dcd271e6be9a




0it [00:00, ?it/s]

,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,21,"The document discusses the potential impacts of geopolitical conflicts, particularly in Israel, on the company's operations and employees, including financial support for affected staff and possible disruptions in product development. While the global supply chain remains stable, ongoing conflicts may pose future challenges. It also highlights risks from climate change, such as extreme weather and regulatory costs, which could increase operational expenses and legal liabilities. Additionally, challenges in integrating acquisitions may hinder growth and product development. The company emphasizes the importance of managing these risks to ensure business stability and growth...."
1,NVIDIA_Report_2024.pdf,34,"In Q3 FY2023, the U.S. government imposed new licensing requirements on exports of A100 and H100 integrated circuits to China, Russia, and certain Middle Eastern countries, resulting in a decline in sales to China from 19% to 14% of total Data Center revenue. The company is seeking alternative, license-free products and has initiated limited shipments to China. Ongoing geopolitical tensions and potential changes in U.S. export controls could negatively impact competitiveness and financial performance. Additionally, macroeconomic challenges like inflation and supply chain issues may affect operations, while the Israel-Hamas conflict is being monitored for potential disruptions due to employee military absences...."
2,NVIDIA_Report_2024.pdf,10,"NVIDIA is committed to matching 100% of its global electricity usage with renewable energy, increasing its renewable energy purchases to 44% in fiscal year 2023, and targeting 67% engagement with its manufacturing suppliers by fiscal year 2026. The company prioritizes energy efficiency in its technology, as shown by its high ranking on the Green500 list, and is developing Earth-2 for climate change predictions. With around 29,600 employees, 83% in technical roles, NVIDIA emphasizes diversity, employee development, and engagement, resulting in a low turnover rate of 2.7% in fiscal year 2024. Its compensation program aligns employee interests with shareholders and includes comprehensive health benefits...."


,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,21,"Table of Contents\nWe are monitoring the impact of the geopolitical conflict in and around Israel on our operations, including the health and safety of our approximately 3,700employees in the region who primarily support the research and development, operations, and sales and marketing of our networking products. Our operatingexpenses in fiscal year 2024 include expenses for financial support to impacted employees and charitable activity. We believe our global supply chain for our\nnetworking products has not experienced any significant impact. Further, in connection with the conflict, a substantial number of our employees in the regionhave been called-up for active military duty in Israel. Accordingly, some of our employees in Israel have been absent for an extended period and they or others..."
1,NVIDIA_Report_2024.pdf,34,"networking products has not experienced any significant impact. Further, in connection with the conflict, a substantial number of our employees in the regionhave been called-up for active military duty in Israel. Accordingly, some of our employees in Israel have been absent for an extended period and they or othersmay continue to be absent, which may cause disruption to our product development or operations. We did not experience any significant impact or expense to\nour business; however, if the conflict is further extended, it could impact future product development, operations, and revenue or create other uncertainty for ourbusiness.\n35..."
2,NVIDIA_Report_2024.pdf,34,"these macroeconomic factors can also impact our supply chain and manufacturing costs, employee wages, costs for capital equipment and value of ourinvestments. Our product and solution pricing generally does not fluctuate with short-term changes in our costs. Within our supply chain, we continuously\nmanage product availability and costs with our vendors. \nIsrael and Hamas Conflict\nWe are monitoring the impact of the geopolitical conflict in and around Israel on our operations, including the health and safety of our approximately 3,700\nemployees in the region who primarily support the research and development, operations, and sales and marketing of our networking products. Our operatingexpenses in fiscal year 2024 include expenses for financial support to impacted employees and charitable activity. We believe our global supply chain for our..."
3,NVIDIA_Report_2024.pdf,34,"Table of Contents\nGlobal Trade\nDuring the third quarter of fiscal year 2023, the USG, announced licensing requirements that, with certain exceptions, impact exports to China (including Hong\nKong and Macau) and Russia of our A100 and H100 integrated circuits, DGX or any other systems or boards which incorporate A100 or H100 integrated circuits.\nIn July 2023, the USG informed us of an additional licensing requirement for a subset of A100 and H100 products destined to certain customers and other\nregions, including some countries in the Middle East.\nIn October 2023, the USG announced new and updated licensing requirements that became effective in our fourth quarter of fiscal year 2024 for exports toChina and Country Groups D1, D4, and D5 (including but not limited to Saudi Arabia, the United Arab Emirates, and Vietnam, but excluding Israel) of our..."
4,NVIDIA_Report_2024.pdf,10,"many workloads because it is optimized for throughput, performance per watt, and certain AI workloads. The energy efficiency of our products is evidenced byour continued strong presence on the Green500 list of the most energy-efficient systems. We powered 24 of the top 30 most energy efficient systems, includingthe top supercomputer, on the Green500 list.\nWe plan to build Earth-2, a digital twin of the Earth on NVIDIA AI and NVIDIA Omniverse platforms. Earth-2 will enable scientists, companies, and policy makersto do ultra-high-resolution predictions of the impact of climate change and explore mitigation and adaptatio

,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,35,"In fiscal year 2024, NVIDIA's revenue reached $60.9 billion, a 126% increase from the previous year. Data Center revenue soared 217% to $47.5 billion, fueled by demand for AI and cloud services. Gaming revenue grew 15% to $10.4 billion, while Professional Visualization and Automotive revenues increased by 1% and 21%, respectively. Gross margin improved to 72.7%, and net income surged 581% to $29.8 billion, despite a slight rise in operating expenses due to employee growth and compensation...."
1,NVIDIA_Report_2024.pdf,49,"For the year ending January 28, 2024, NVIDIA Corporation reported a revenue of $60.92 billion, up from $26.97 billion in 2023. The cost of revenue was $16.62 billion, resulting in a gross profit of $44.30 billion. Operating expenses increased to $11.33 billion, yielding an operating income of $32.97 billion. After taxes, the net income was $29.76 billion, equating to a diluted net income per share of $11.93...."
2,NVIDIA_Report_2024.pdf,38,"For the year ending January 28, 2024, the company experienced substantial financial growth, with revenue up 126% to $60.9 billion and gross profit at 72.7%. Operating expenses decreased to 18.6%, resulting in an operating income of 54.1%. Net income rose to 48.9%, a significant increase from 16.2%. The Compute & Networking segment saw a 215% revenue boost, largely from Data Center sales and NVIDIA Hopper GPU shipments, while Graphics revenue grew by 14%, fueled by a 15% rise in Gaming. Overall, both segments reported strong operating income growth...."


...

,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,87,"EXHIBIT 31.1\nCERTIFICATION\n \nI, Jen-Hsun Huang, certify that:\n \n1. I have reviewed this Annual Report on Form 10-K of NVIDIA Corporation;\n2. Based on my knowledge, this report does not contain any untrue statement of a material fact or omit to state a material fact necessary to\nmake the statements made, in light of the circumstances under which such statements were made, not misleading with respect to the period\ncovered by this report;\n \n3. Based on my knowledge, the financial statements, and other financial information included in this report, fairly present in all material respects\nthe financial condition, results of operations and cash flows of the registrant as of, and for, the periods presented in this report;\n \n4. The registrant's other certifying officer(s) and I are responsible for establishing and maintaining disclosure controls and procedures (as..."
1,NVIDIA_Report_2024.pdf,83,"Signatures\nPursuant to the requirements of Section 13 or 15(d) of the Securities Exchange Act of 1934, the Registrant has duly caused this report to be signed on its behalfby the undersigned, thereunto duly authorized, on February 21, 2024.\nNVIDIA Corporation\nBy: /s/ Jen-Hsun Huang \n Jen-Hsun Huang\n President and Chief Executive Officer\nPower of Attorney\nKNOW ALL PERSONS BY THESE PRESENTS, that each person whose signature appears below constitutes and appoints Jen-Hsun Huang and Colette M.Kress, and each or any one of them, his true and lawful attorney-in-fact and agent, with full power of substitution and resubstitution, for him and in his name,..."
2,NVIDIA_Report_2024.pdf,89,"EXHIBIT 32.1\n \nCERTIFICATION \n \nPursuant to the requirement set forth in Rule 13a-14(b) of the Securities Exchange Act of 1934, as amended (the “Exchange Act”), and\nSection 1350 of Chapter 63 of Title 18 of the United States Code (18 U.S.C. § 1350), Jen-Hsun Huang, the President and Chief Executive\nOfficer of NVIDIA Corporation (the “Company”), hereby certifies that, to the best of his knowledge:\n1. The Company's Annual Report on Form 10-K for the year ended January 28, 2024, to which this Certification is attached as Exhibit 32.1 (the\n“Periodic Report”), fully complies with the requirements of Section 13(a) or Section 15(d) of the Exchange Act; and\n2. The information contained in the Periodic Report fairly presents, in all material respects, the financial condition of the Company at the end of\nthe period covered by the Periodic Report and results of operations of the Company for the period covered by the Periodic Report.\nDate: February 21, 2024\n \n \n/s/ JEN-HSUN HUANG..."
3,NVIDIA_Report_2024.pdf,89,"the period covered by the Periodic Report and results of operations of the Company for the period covered by the Periodic Report.\nDate: February 21, 2024\n \n \n/s/ JEN-HSUN HUANG\nJen-Hsun Huang \nPresident and Chief Executive Officer\n \n \nA signed original of this written statement required by Section 906 of 18 U.S.C. § 1350 has been provided to NVIDIA Corporation and will be\nretained by NVIDIA Corporation and furnished to the Securities and Exchange Commission or its staff upon request.\n \nThis certification accompanies the Form 10-K to which it relates, is not deemed filed with the Securities and Exchange Commission and is\nnot to be incorporated by reference into any filing of the Company under the Securities Act of 1933, as amended, or the Exchange Act (whether\nmade before or after the date of the Form 10-K), irrespective of any general incorporation language contained in such filing...."


,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,83,"On February 21, 2024, NVIDIA Corporation submitted a report, signed by CEO Jen-Hsun Huang, in accordance with the Securities Exchange Act of 1934. The report includes a Power of Attorney designating Huang and Colette M.Kress as attorneys-in-fact to sign amendments and file related documents with the SEC...."
1,NVIDIA_Report_2024.pdf,89,"Jen-Hsun Huang, CEO of NVIDIA, certifies that the company's Annual Report on Form 10-K for the year ending January 28, 2024, complies with the Securities Exchange Act and accurately reflects its financial status. Dated February 21, 2024, the certification will be retained by NVIDIA and provided to the SEC upon request, but is not filed with the SEC or included in other filings...."
2,NVIDIA_Report_2024.pdf,87,"Jen-Hsun Huang, CEO of NVIDIA, certifies the accuracy of the company's Annual Report on Form 10-K, affirming that it contains no misleading statements and fairly represents the financial condition. He outlines management's responsibilities for disclosure controls and internal financial reporting, and discloses any significant deficiencies or fraud. The certification is dated February 21, 2024...."


,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,83,"Signatures\nPursuant to the requirements of Section 13 or 15(d) of the Securities Exchange Act of 1934, the Registrant has duly caused this report to be signed on its behalfby the undersigned, thereunto duly authorized, on February 21, 2024.\nNVIDIA Corporation\nBy: /s/ Jen-Hsun Huang \n Jen-Hsun Huang\n President and Chief Executive Officer\nPower of Attorney\nKNOW ALL PERSONS BY THESE PRESENTS, that each person whose signature appears below constitutes and appoints Jen-Hsun Huang and Colette M.Kress, and each or any one of them, his true and lawful attorney-in-fact and agent, with full power of substitution and resubstitution, for him and in his name,..."
1,NVIDIA_Report_2024.pdf,89,"EXHIBIT 32.1\n \nCERTIFICATION \n \nPursuant to the requirement set forth in Rule 13a-14(b) of the Securities Exchange Act of 1934, as amended (the “Exchange Act”), and\nSection 1350 of Chapter 63 of Title 18 of the United States Code (18 U.S.C. § 1350), Jen-Hsun Huang, the President and Chief Executive\nOfficer of NVIDIA Corporation (the “Company”), hereby certifies that, to the best of his knowledge:\n1. The Company's Annual Report on Form 10-K for the year ended January 28, 2024, to which this Certification is attached as Exhibit 32.1 (the\n“Periodic Report”), fully complies with the requirements of Section 13(a) or Section 15(d) of the Exchange Act; and\n2. The information contained in the Periodic Report fairly presents, in all material respects, the financial condition of the Company at the end of\nthe period covered by the Periodic Report and results of operations of the Company for the period covered by the Periodic Report.\nDate: February 21, 2024\n \n \n/s/ JEN-HSUN HUANG..."
2,NVIDIA_Report_2024.pdf,89,"the period covered by the Periodic Report and results of operations of the Company for the period covered by the Periodic Report.\nDate: February 21, 2024\n \n \n/s/ JEN-HSUN HUANG\nJen-Hsun Huang \nPresident and Chief Executive Officer\n \n \nA signed original of this written statement required by Section 906 of 18 U.S.C. § 1350 has been provided to NVIDIA Corporation and will be\nretained by NVIDIA Corporation and furnished to the Securities and Exchange Commission or its staff upon request.\n \nThis certification accompanies the Form 10-K to which it relates, is not deemed filed with the Securities and Exchange Commission and is\nnot to be incorporated by reference into any filing of the Company under the Securities Act of 1933, as amended, or the Exchange Act (whether\nmade before or after the date of the Form 10-K), irrespective of any general incorporation language contained in such filing...."
3,NVIDIA_Report_2024.pdf,87,"EXHIBIT 31.1\nCERTIFICATION\n \nI, Jen-Hsun Huang, certify that:\n \n1. I have reviewed this Annual Report on Form 10-K of NVIDIA Corporation;\n2. Based on my knowledge, this report does not contain any untrue statement of a material fact or omit to state a material fact necessary to\nmake the statements made, in light of the circumstances under which such statements were made, not misleading with respect to the period\ncovered by this report;\n \n3. Based on my knowledge, the financial statements, and other financial information included in this report, fairly present in all material respects\nthe financial condition, results of operations and cash flows of the registrant as of, and for, the periods presented in this report;\n \n4. The registrant's other certifying officer(s) and I are responsible for establishing and maintaining disclosure controls and procedures (as..."


,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,68,"As of January 28, 2024, NVIDIA Corporation reported total property and equipment valued at $7.423 billion, with a net value of $3.914 billion after $3.509 billion in accumulated depreciation. Key assets include land ($218 million), buildings ($1.816 billion), and equipment ($5.200 billion). Depreciation expenses for fiscal years 2024, 2023, and 2022 were $894 million, $844 million, and $611 million, respectively. Additionally, other assets amounted to $4.500 billion, primarily from prepaid supply agreements ($2.458 billion) and investments in non-affiliated entities ($1.546 billion)...."
1,NVIDIA_Report_2024.pdf,52,"The consolidated statements of shareholders' equity for NVIDIA Corporation from January 31, 2021, to January 28, 2024, show significant changes in equity components. Key points include:\n\n- **2021**: 2,479 million shares and total equity of $16.893 billion, with a net income of $9.752 billion.\n- **2022**: Shares rose to 2,506 million and total equity increased to $26.612 billion, despite a lower net income of $4.368 billion and treasury stock retirement.\n- **2023**: Shares decreased to 2,466 million, total equity fell to $22.101 billion, but net income surged to $29.760 billion.\n- **2024**: As of January 28, 2024, shares slightly declined to 2,464 million, with total equity reaching $42.978 billion, driven by continued net income growth.\n\nOverall, the statements reflect NVIDIA's strong financial performance and evolving shareholder equity over the three years...."
2,NVIDIA_Report_2024.pdf,43,"The report addresses financial statement risks from interest rate and foreign exchange fluctuations. The company mitigates foreign exchange risk by conducting sales in U.S. dollars and using forward contracts, resulting in minimal impact on net income for fiscal years 2024 and 2023. A 10% strengthening of the U.S. dollar could decrease accumulated other comprehensive income by $116 million and $112 million, respectively, while adverse currency changes could reduce income before taxes by $60 million and $36 million. Management confirmed the effectiveness of their disclosure and internal financial reporting controls as of January 28, 2024, with no significant changes noted. An audit by PricewaterhouseCoopers LLP validated the effectiveness of these internal controls...."


,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,68,"(C) Construction in process represents assets that are not available for their intended use as of the balance sheet date.\nDepreciation expense for fiscal years 2024, 2023, and 2022 was $894 million, $844 million, and $611 million, respectively.\nAccumulated amortization of leasehold improvements and finance leases was $400 million and $327 million as of January 28, 2024 and January 29, 2023,respectively.\nProperty, equipment and intangible assets acquired by assuming related liabilities during fiscal years 2024, 2023, and 2022 were $170 million, $374 million, and\n$258 million, respectively.\n Jan 28, 2024 Jan 29, 2023\nOther assets: (In millions)\nPrepaid supply and capacity agreements (1) $ 2,458 $ 2,989 \nInvestments in non-affiliated entities 1,546 299 \nPrepaid royalties 364 387 \nOther 132 145 \nTotal other assets $ 4,500 $ 3,820..."
1,NVIDIA_Report_2024.pdf,68,"Table of Contents\nNVIDIA Corporation and SubsidiariesNotes to the Consolidated Financial Statements(Continued)\n Jan 28, 2024 Jan 29, 2023 EstimatedUseful Life\n(In millions) (In years)\nProperty and Equipment:\nLand $ 218 $ 218 (A)\nBuildings, leasehold improvements, and furniture 1,816 1,598 (B)\nEquipment, compute hardware, and software 5,200 4,303 3-7\nConstruction in process 189 382 (C)\nTotal property and equipment, gross 7,423 6,501 \nAccumulated depreciation and amortization (3,509) (2,694) \nTotal property and equipment, net $ 3,914 $ 3,807 \n(A) Land is a non-depreciable asset.\n(B) The estimated useful lives of our buildings are up to thirty years. Leasehold improvements and finance leases are amortized based on the lesser of either the asset’s estimated useful life or theexpected remaining lease term.\n(C) Construction in process represents assets that are not available for their intended use as of the balance sheet date...."
